**Tabla de contenido**

- [Preparación entorno spark](#Preparacion-entorno-spark)
- [Lectura de datos](#Lectura-de-datos)
- [Limpieza y transformacion](#Limpieza-y-transformacion)
    - [Duplicados](#Duplicados)
    - [Convertir columnas a su respectivo formato](#Convertir-columnas-a-su-respectivo-formato)
    - [Valores faltantes](#Valores-faltantes)

# Preparacion entorno spark

Vamos a preparar el entorno de trabajo con las siguientes configuraciones:

1. Importaciones:

- `SparkContext:` Punto de entrada principal para funcionalidades de Spark
- `SparkSession:` Interfaz unificada para trabajar con datos en Spark


2. `sc = SparkContext('local')`:

- Crea un contexto de Spark en modo local. 
- Significa que Spark correrá en tu máquina, usando todos los núcleos disponibles.
- Ideal para desarrollo y pruebas
3. `spark = SparkSession(sc):`

- Crea una sesión de Spark usando el contexto previamente creado
- Permite trabajar con DataFrames y realizar operaciones SQL
- Es la forma moderna de interactuar con Spark (reemplaza RDDs antiguos)

In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local')
spark = SparkSession(sc)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/22 17:53:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Lectura de datos

Vamos a leer el dataframe y lo almacenaremos en el cache por la siguientes razones:
1. Estaremos llamando el dataframe muchas veces ya que lo vamos a limpiar. Si no cacheas, Spark recalculará desde el origen en cada paso intermedio (ineficiente).
2. El dataframe es pequeño, por lo cual cabe en memoria.

`Beneficios`

- Velocidad: Operaciones como dropDuplicates() o groupBy() serán más rápidas al evitar recalcular desde el origen.

- Simplicidad: No necesitas preocuparte por repetir lecturas del archivo.

- Debugging: Puedes revisar resultados intermedios con show() sin penalización de rendimiento.

`¿Cuándo NO cachear?`

- Si el DataFrame solo se usa una vez.
- Si haces una sola acción después de todas las transformaciones.

`Para DataFrames grandes que no caben en memoria pero requieren reutilización:̣`

- Usa MEMORY_AND_DISK como nivel de persistencia.

- Reparticiona para manejar trozos más pequeños.

- Monitoriza el uso de recursos en la UI de Spark.

In [2]:
import os

file_path = lambda file: os.path.join(os.getcwd(),'data',file)
df = spark.read.csv(file_path('Table_Store.csv'),sep=',',header=True)
df.show(10)

+--------+--------------+----------------+--------+----------+--------------------+
|Store_ID|          City|       Open_Date|Zip_Code|     Phone|               Email|
+--------+--------------+----------------+--------+----------+--------------------+
|       1| *San Antonio*|24/07/2003 21:45|   78015|2136875667| *store1@retail.com*|
|       2| @San Antonio!| 27/10/2005 7:01|   78201|2135167236| @store2@retail.com!|
|       3| #San Antonio#|01/02/2007 16:59|   78112|2123501192|#store3@walmart.com#|
|       4|     &Houston&| 17/07/2004 0:28|   77001|3129778178|&store4@walmart.com&|
|       5|     !Houston!| 23/04/2009 1:08|   77002|7136191883|  !store5@store.com!|
|       6| $Los Angeles$|10/08/2006 19:42|   90001|2127216342|  $store6@store.com$|
|       7|     %Phoenix%| 16/10/2007 8:02|   60601|7138018755|  %store7@store.com%|
|       8|   #San Diego#| 10/03/2002 5:40|   94023|2125445446| #store8@retail.com#|
|       9|    @New York@| 24/07/2007 4:31|   10001|3122242101|  @store9@stor

Veamos la estructura del dataframe

In [3]:
df.printSchema()

root
 |-- Store_ID: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Open_Date: string (nullable = true)
 |-- Zip_Code: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Email: string (nullable = true)



Podemos observar lo siguiente:
1. Todas las columnas son de tipo String
1. Todas las columnas permiten valores nulos `nullable = true`

# Limpieza y transformacion

La limpieza y transformación de los datos la vamos a realizar siguiendo los siguientes pasos:

1. Consultar la cantidad de registros dúplicados
2. Convertir las columnas a su respectivo formato.
3. Consultar la cantidad de valores faltantes
4. Eliminar caracteres especiales


## Duplicados

In [4]:
duplicados = df.groupBy(df.columns).count().filter('count > 1').count()
print(f"Existen un total de {duplicados} registros dúplicados")
df= df.dropDuplicates() # elimina los dúplicados

Existen un total de 0 registros dúplicados


## Convertir columnas a su respectivo formato

In [5]:
from pyspark.sql.functions import to_timestamp, col

columns_to_num = ["Store_ID"] # columnas a convertir a entero
for col_name in columns_to_num:
    df = df.withColumn(col_name, df[col_name].cast("int"))

# convierte la clumna Open_Date a datetime 
df = df.withColumn("Open_Date", to_timestamp(col("Open_Date"), "dd/MM/yyyy H:mm"))
df = df.sort("Store_ID")
df.printSchema()

root
 |-- Store_ID: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Open_Date: timestamp (nullable = true)
 |-- Zip_Code: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Email: string (nullable = true)



In [6]:
df.show(10)

+--------+--------------+-------------------+--------+----------+--------------------+
|Store_ID|          City|          Open_Date|Zip_Code|     Phone|               Email|
+--------+--------------+-------------------+--------+----------+--------------------+
|       1| *San Antonio*|2003-07-24 21:45:00|   78015|2136875667| *store1@retail.com*|
|       2| @San Antonio!|2005-10-27 07:01:00|   78201|2135167236| @store2@retail.com!|
|       3| #San Antonio#|2007-02-01 16:59:00|   78112|2123501192|#store3@walmart.com#|
|       4|     &Houston&|2004-07-17 00:28:00|   77001|3129778178|&store4@walmart.com&|
|       5|     !Houston!|2009-04-23 01:08:00|   77002|7136191883|  !store5@store.com!|
|       6| $Los Angeles$|2006-08-10 19:42:00|   90001|2127216342|  $store6@store.com$|
|       7|     %Phoenix%|2007-10-16 08:02:00|   60601|7138018755|  %store7@store.com%|
|       8|   #San Diego#|2002-03-10 05:40:00|   94023|2125445446| #store8@retail.com#|
|       9|    @New York@|2007-07-24 04:31:0

## Valores faltantes

In [7]:
df = df.cache()
df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- InMemoryTableScan [Store_ID#76, City#18, Open_Date#77, Zip_Code#20, Phone#21, Email#22]
      +- InMemoryRelation [Store_ID#76, City#18, Open_Date#77, Zip_Code#20, Phone#21, Email#22], StorageLevel(disk, memory, deserialized, 1 replicas)
            +- AdaptiveSparkPlan isFinalPlan=false
               +- Sort [Store_ID#76 ASC NULLS FIRST], true, 0
                  +- Exchange rangepartitioning(Store_ID#76 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=212]
                     +- HashAggregate(keys=[Zip_Code#20, Email#22, City#18, Open_Date#19, Phone#21, Store_ID#17], functions=[])
                        +- Exchange hashpartitioning(Zip_Code#20, Email#22, City#18, Open_Date#19, Phone#21, Store_ID#17, 200), ENSURE_REQUIREMENTS, [plan_id=209]
                           +- HashAggregate(keys=[Zip_Code#20, Email#22, City#18, Open_Date#19, Phone#21, Store_ID#17], functions=[])
                              +- FileScan csv [